Решим новую задачу: добавим временны́е метки ко всем записям; для каждой записи каждой таблицы будем сохранять время создания записи и время последнего изменения.

Для этого добавим во все таблицы поля `created_at` (время создания) и `modified_at` (время последнего изменения):

In [ ]:
from django.db import models

...

class VideoProduct(BaseModel):
    ...
    created_at = models.DateTimeField(auto_now_add=True)
    modified_at = models.DateTimeField(auto_now_add=False, auto_now=True)

Во всех моделях надо описать совершенно одинаковые поля; но дублирование кода — это дурной тон и нарушение принципа DRY. Повторов можно избежать: для этого в Django ORM применяют наследование от абстрактных моделей. 

Практический смысл абстрактной модели в том, что в ней можно задать определённый набор свойств, затем унаследовать от неё другие модели — и все эти модели получат свойства, заданные в абстрактной модели-родителе.

Чтобы не запутаться в назначении классов, в названиях абстрактных моделей обычно используют слово Model: *BaseModel*, *CartModel*, *ProductModel*.

Опишем первую абстрактную модель — `BaseModel`; в ней определим класс `Meta` — внутренний класс, в котором описывают дополнительные настройки модели. 

Класс `Meta` необязательный, он требуется не во всех классах, но в абстрактных моделях он необходим — именно в `Meta` модель объявляется абстрактной:

In [ ]:
class BaseModel(models.Model):
    """
    Абстрактная модель. 
    Добавляет к модели дату создания и последнего изменения. 
    """
    # Параметр auto_now_add=True означает
    # "при СОЗДАНИИ записи автоматически записывать в это поле текущее время".
    created_at = models.DateTimeField(auto_now_add=True)
    # Параметр auto_now=True означает
    # "при ИЗМЕНЕНИИ записи автоматически записывать в это поле текущее время".
    modified_at = models.DateTimeField(auto_now_add=False, auto_now=True)

    # С помощью необязательного внутреннего класса Meta можно добавить
    # к модели дополнительные настройки. 
    class Meta:
        # Эта строка объявляет модель абстрактной:
        abstract = True

Унаследуем `VideoProduct` от `BaseModel` — и в модели `VideoProduct` не придётся явно описывать поля `created_at` и `modified_at`: 

In [ ]:
...

class VideoProduct(BaseModel):
    title = models.CharField(max_length=128)

Python поддерживает множественное наследование: можно создать несколько абстрактных моделей и наследоваться от них, просто перечислив их имена через запятую.

In [ ]:
...

class BaseModel(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)
    modified_at = models.DateTimeField(auto_now_add=False, auto_now=True)

    class Meta:
        abstract = True

class CommonInfoBaseModel(models.Model):
    name = models.CharField(max_length=128)
    description = models.TextField()
    
    class Meta:
        abstract = True

# В этой модели явно описано только поле bottle_volume, но за счёт наследования
# модель будет содержать и поля 
# created_at, modified_at, name и description:
class BottledMilk(BaseModel, CommonInfoBaseModel):
    bottle_volume = models.IntegerField()

# А тут явным образом описано только поле weight; 
# остальные поля будут также унаследованы от базовых моделей:
class Cheese(BaseModel, CommonInfoBaseModel):
    weight = models.IntegerField()

# Для этой модели нужны поля только одного базового класса:
class Employee(BaseModel):
    first_name = models.CharField(max_length=128)
    second_name = models.CharField(max_length=128)
    role = models.CharField(max_length=128)


Одну абстрактную модель можно наследовать от другой. При этом наследники абстрактных классов не будут по умолчанию абстрактными. Для этого в модели-наследнике тоже нужно указать атрибут `abstract = True`:

In [ ]:
class BaseModel(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)
    modified_at = models.DateTimeField(auto_now_add=False, auto_now=True)

    class Meta:
        abstract = True

class CommonInfoBaseModel(BaseModel):
    name = models.CharField(max_length=128)
    description = models.TextField()
    
    class Meta:
        abstract = True

Наследование от абстрактных моделей — хорошая практика, но в работе необходимо следовать здравому смыслу и создавать абстрактные модели лишь в тех случаях, когда они действительно нужны, например — когда в нескольких классах повторяется один и тот же код.

***
## Хозяйке на заметку

Много интересного про `class Meta` можно [увидеть в документации](https://docs.djangoproject.com/en/3.2/ref/models/options/).